# Feature Engineering

Here, we explore some features of interest. Our features fall under a few general categories:

- <u>**Basic user features/stats**</u>: features we have already or that can be easily derived
    - reply ratio (% of comments that are replies instead of main comments)
    - comment count
    - user creation date

- <u>**User behavior features**</u>: features that indicate behavior across videos
    - average comment latency (how long ago from video publish do they comment?)
    - std deviation in comment latency (how varied are their response latencies?)
    - comment hours (what hours of the day do they comment?)
    - channel diversity (do they comment across multiple channels? do they comment across perceived 'party lines'?)

- <u>**Graph features**</u>: features that explore patterns in clusters
    - co-commenters: groups of users who comment on the same videos
    - temporal clustering: groups commenting within minutes

- <u>**Content features**</u>: features that explore the text content itself
    - comment length
    - std deviation in comment length
    - copy amount (how many times they copy-paste their comment(s))
    - sentiment analysis (TODO later)

## 0. Setup (and Postgres)

In [ ]:
import seaborn as sns
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from pprint import pprint

import sys
import os
from pathlib import Path

project_root = Path(os.getcwd()).parent if 'notebooks' in os.getcwd() else Path(os.getcwd())
sys.path.insert(0, str(project_root)) # Make root directory the cwd

from dotenv import load_dotenv
load_dotenv()

# Override Postgres env variables for local runtime
os.environ["POSTGRES_PORT"] = os.getenv("POSTGRES_PORT_LOCAL", 5555)
os.environ["POSTGRES_HOST"] = "localhost"

from src.data.psql import Psql

psql_client = Psql()

## 1. Basic User Features

In [16]:
psql_client.rollback()

In [ ]:
# Reply Ratio

query = """
SELECT c.commenterId,
    COUNT(*) FILTER (WHERE c.threadId IS NOT NULL) AS reply_count,
    COUNT(*) FILTER (WHERE c.threadId IS NULL) AS toplevel_count,
    CASE
        WHEN COUNT(*) FILTER (WHERE threadId IS NULL) = 0 THEN NULL
        ELSE COUNT(*) FILTER (WHERE threadId IS NOT NULL)::float /
            COUNT(*) FILTER (WHERE threadId IS NULL)
    END AS reply_ratio
FROM Yt.Comments AS c
GROUP BY c.commenterId
ORDER BY reply_ratio ASC NULLS LAST
LIMIT 10;
"""

pd.DataFrame(psql_client.query(query), columns=["userId", 
                                                "numReplies", 
                                                "numComments", 
                                                "replyRatio"])

,userId,numReplies,numComments,replyRatio
0,UC001JnHRDXEkrVbocOu30NA,0,2,0.0
1,UC001R2x2-Sc6LrJGjN8c-vA,0,1,0.0
2,UC003EBy9QKLw7IRcBnbdv9w,0,1,0.0
3,UC004dztqUCdJG1VotvJeWaA,0,1,0.0
4,UC00A3kA-rQZ8eA8VHI-pRAA,0,1,0.0
5,UC_00ACj8nUkrd8Eo50rwnTQ,0,1,0.0
6,UC00AqVXniD4MBnsesB3wKGw,0,2,0.0
7,UC00AZcJ0gwabUdxfwZwB4Eg,0,1,0.0
8,UC00e41dgt0zrLuas2SYMPtg,0,3,0.0
9,UC001ABAlK_3Idd3mu_0NwQA,0,1,0.0


In [23]:
# Comment counts

query = """
SELECT c.commenterId, COUNT(c.commenterId) AS commentCount
FROM Yt.Comments AS c
GROUP BY c.commenterId
ORDER BY commentCount DESC
LIMIT 20;
"""

pd.DataFrame(psql_client.query(query), columns=["userId", "commentCount"])

,userId,commentCount
0,UCj-MdOznJW2YvAoC3tiREvg,78
1,UCqhnAU_fXoUKINKnxijG0CQ,67
2,UCHdAu6riUiqViLO3736o_8g,60
3,UChd2niRs5P2rH1SZdwT9g2w,59
4,UCyQwpzdUBWaUjStqO9qrX9Q,58
5,UCTRzwbjZxwzSwuIt9GfqqHg,56
6,UCMLkTrKuK8a9PwL4x51dG3g,55
7,UCJmuTfLf2YxbTLVaKdVQeGw,55
8,UCDYQFG5CkxQFKG-cgD_vpBg,54
9,UC-3Nj6oYW1QqHjBmm3iL-oQ,54


In [ ]:
# User creation date

query = """
SELECT c.commenterId, COUNT(c.commenterId) AS commentCount, u.createDate
FROM Yt.Comments AS c
JOIN Yt.Users AS u
    ON c.commenterId = u.userId
GROUP BY c.commenterId, u.createDate
ORDER BY u.createDate DESC
LIMIT 20;
"""

pd.DataFrame(psql_client.query(query), columns=["userId", 
                                                "commentCount", 
                                                "accountCreationDate"])

,userId,commentCount,accountCreationDate
0,UCI6GmAmJcEHwco9gyU3Y4iw,1,2025-10-14 18:15:14.434315
1,UCvJkitF2wVbQQoNd223wo8A,2,2025-10-14 17:40:15.795096
2,UCu2wQb5xZ-w75PFPXT-FO2Q,7,2025-10-14 17:13:28.914567
3,UCTEel0tNZZOs0rrenWfQw-g,1,2025-10-14 17:09:38.827790
4,UCiVtdPxIoIKifWZY2d6SCqA,1,2025-10-14 17:08:04.908882
5,UC1AtFxkWVd6-jNvCagbdWZg,2,2025-10-14 17:06:29.916560
6,UCBYlck_yCUZin7s-zqz4zPA,1,2025-10-14 17:03:25.327494
7,UCYfvfWa77eT-vP2Uf6Xnu5w,1,2025-10-14 17:02:55.312908
8,UCaRbS3zmkeV2jYtoEV2nOhw,2,2025-10-14 17:02:53.252181
9,UC-c1v2PSBwxmtyOKrLFNEkQ,3,2025-10-14 17:02:51.877294


## 2. User Behavior Features

In [63]:
psql_client.rollback()

### 2.1 Avg, stddev comment latency, sorted by avg

In [61]:
# Average and stddev comment latency, sorted by avg

query = """
SELECT c.commenterId, COUNT(c.commenterId) AS commentCount, 
    AVG(EXTRACT(EPOCH FROM(c.publishDate - v.publishDate))) AS avgCommentLatency,
    STDDEV(EXTRACT(EPOCH FROM (c.publishDate - v.publishDate))) AS stddevCommentLatency
FROM Yt.Comments AS c
JOIN Yt.Videos AS v
    ON c.videoId = v.videoId
GROUP BY c.commenterId
HAVING COUNT(c.commenterId) > 2
    AND AVG(EXTRACT(EPOCH FROM (c.publishDate - v.publishDate))) > 0
    AND STDDEV(EXTRACT(EPOCH FROM (c.publishDate - v.publishDate))) IS NOT NULL
ORDER BY avgCommentLatency ASC
LIMIT 20;
"""

pd.DataFrame(psql_client.query(query), columns=["userId",
                                                "commentCount",
                                                "avgLatency", 
                                                "stddevLatency"])

,userId,commentCount,avgLatency,stddevLatency
0,UCGGFLahyrUCpBw-zwx-ADPQ,3,31.6666666666666667,28.0059517483932928
1,UC9r28oLfrnCVQVX-ri7gNCA,3,65.6666666666666667,22.1208800307160776
2,UCmcDnAYVYNVcbCysQQIagEA,3,92.0000000000000000,22.0000000000000000
3,UC-c1v2PSBwxmtyOKrLFNEkQ,3,101.6666666666666667,5.1316014394468841
4,UCi9FpvBI75U6gS1Tb3LMnDQ,4,105.7500000000000000,21.9905282640807275
5,UCW-7QKBkIX37OlkOA59UKCQ,3,120.3333333333333333,77.1837634048336188
6,UCJavIC8f_62tIDHQBi9toCw,3,122.3333333333333333,50.0533049191892356
7,UCk8HRRkc07FGZ6CpiyzsFVQ,3,136.0000000000000000,98.2293235240882640
8,UCEJTYokjLHEptRLGcfYX-Ww,3,144.0000000000000000,133.367912182804
9,UC6oW5BKYvEM8wH_3Yjtp4xw,3,173.0000000000000000,101.4248490262618039


### 2.2 Avg, stddev comment latency, sorted by stddev

Having low standard deviations in comment latency across multiple comments may suggest a more systematic pattern in the timing of the comments in relation to the upload time of the video.

We see that the first user on the list below exhibits a standard deviation in latency of 0 across 3 comments, which suggest exact timing

In [56]:
# Average and stddev comment latency, sorted by stddev

query = """
SELECT c.commenterId, COUNT(c.commenterId) AS commentCount, 
    AVG(EXTRACT(EPOCH FROM(c.publishDate - v.publishDate))) AS avgCommentLatency,
    STDDEV(EXTRACT(EPOCH FROM (c.publishDate - v.publishDate))) AS stddevCommentLatency
FROM Yt.Comments AS c
JOIN Yt.Videos AS v
    ON c.videoId = v.videoId
GROUP BY c.commenterId
HAVING COUNT(c.commenterId) > 2
    AND AVG(EXTRACT(EPOCH FROM (c.publishDate - v.publishDate))) > 0
    AND STDDEV(EXTRACT(EPOCH FROM (c.publishDate - v.publishDate))) IS NOT NULL
ORDER BY stddevCommentLatency ASC
LIMIT 20;
"""

pd.DataFrame(psql_client.query(query), columns=["userId", "commentCount", "avgLatency", "stddevLatency"])

,userId,commentCount,avgLatency,stddevLatency
0,UCTDo2HyelBLvbgI8NevWywA,3,90601.000000000000,0
1,UC-c1v2PSBwxmtyOKrLFNEkQ,3,101.6666666666666667,5.1316014394468841
2,UChu6Pf2fZ8ctN612i4hKSdw,5,195232.400000000000,6.8044103344816000
3,UCVnN2J6dytQ4O4MmsD1BXGg,4,2282.5000000000000000,7.0000000000000000
4,UCu42_JfJ8bMEHl_h3CXXw-A,3,52612.000000000000,10.1488915650922195
5,UC69i80dSn86ufaz9fK64qFw,3,6813.0000000000000000,13.1148770486040013
6,UCb9cobEB0voDWO2wXuHkWnw,3,6347.6666666666666667,14.6401275039984994
7,UCJBkt9-oZdjebUNjUBcqt8A,9,52763.666666666667,14.7478812037526258
8,UClOTL6fi7TkC1NSyN43rRvA,3,7640.0000000000000000,15.0000000000000000
9,UC4faCSVtci8MRLMrYJnQDfw,3,5131.6666666666666667,16.5025250593154178


### 2.3 Circular features for comment times

Since hours of the day is cyclical, we want to represent hour 0 vs hour 23 as closely related to each other. Thus, we introduct circular statistics (using sine and cosine functions) to measure this

In [68]:
query = """
SELECT c.commenterId, EXTRACT(HOUR FROM c.publishDate) AS comment_hour
FROM Yt.Comments AS c;
"""

comment_hours_df = pd.DataFrame(psql_client.query(query), columns=["userId",
                                                                   "hour"])
comment_hours_df["hour"] = comment_hours_df["hour"].astype(float)
comment_hours_df.head()

,userId,hour
0,UCoHwVlGCKCfsYh_l9Y9mCPA,18.0
1,UCT4h4tL5nDSO1rmkCXxitpQ,18.0
2,UCKQoepgoXPPXCm4EXBZ0RHw,18.0
3,UCu2wQb5xZ-w75PFPXT-FO2Q,18.0
4,UCzsiCl8VmXd_yvnNRZtUubA,18.0


In [73]:
comment_hours_df["radians"] = comment_hours_df["hour"] * (2 * np.pi / 24)
comment_hours_df["sin_hour"] = np.sin(comment_hours_df["radians"])
comment_hours_df["cos_hour"] = np.cos(comment_hours_df["radians"])
comment_hours_df.head()

,userId,hour,radians,sin_hour,cos_hour
0,UCoHwVlGCKCfsYh_l9Y9mCPA,18.0,4.712389,-1.0,-1.836970e-16
1,UCT4h4tL5nDSO1rmkCXxitpQ,18.0,4.712389,-1.0,-1.836970e-16
2,UCKQoepgoXPPXCm4EXBZ0RHw,18.0,4.712389,-1.0,-1.836970e-16
3,UCu2wQb5xZ-w75PFPXT-FO2Q,18.0,4.712389,-1.0,-1.836970e-16
4,UCzsiCl8VmXd_yvnNRZtUubA,18.0,4.712389,-1.0,-1.836970e-16


In [78]:
sin_mean = np.mean(np.sin(comment_hours_df["radians"]))
cos_mean = np.mean(np.cos(comment_hours_df["radians"]))

circular_mean_rad = np.arctan2(sin_mean, cos_mean)
circular_mean_hour = (circular_mean_rad * 24 / (2 * np.pi)) % 24
circular_mean_hour

np.float64(17.597526138485566)

In [84]:
def get_hour_difference_from_mean(hr, circular_mean_hour):
    return min((hr - circular_mean_hour) % 24, (circular_mean_hour - hr) % 24)